In [18]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys, os
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('../util'))
# sys.path.append(os.path.abspath('../'))

%matplotlib inline

In [19]:
data_path = Path('../data/')
# data_genre = data_path / Path('data_w_genres.csv')
df_path= data_path / Path('data_split_by_artist.csv')

CSV file contains float columns as features and using artist as a label for multiclass prediction.
If a song was produced by >1 artist the row has been split into mulitple rows so that each row only
has one artist.

In [20]:
df = pd.read_csv(df_path,index_col=0)
df.head()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,artists
0,0.995,0.708,158648,0.1950,0,0.563,10,0.1510,-12.428,1,0,0.0506,118.469,0.7790,Carl Woitschach
1,0.994,0.379,282133,0.0135,0,0.901,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767,Robert Schumann
2,0.994,0.379,282133,0.0135,0,0.901,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767,Vladimir Horowitz
3,0.604,0.749,104300,0.2200,0,0.000,5,0.1190,-19.924,0,0,0.9290,107.177,0.8800,Seweryn Goszczyński
4,0.995,0.781,180760,0.1300,0,0.887,1,0.1110,-14.734,0,0,0.0926,108.003,0.7200,Francisco Canaro


In [21]:
artists_names = df.artists

In [22]:
index_artist = dict(((i, artist) for i,artist in  enumerate(artists_names.unique())))
artist_index = dict((val, key) for key,val in index_artist.items() )

In [23]:
df = df.drop(['artists'], axis =1)

In [24]:
df = (df-df.min())/(df.max()-df.min())
df.head()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,0.998996,0.716599,0.028442,0.1950,0.0,0.563,0.909091,0.1510,0.745000,1.0,0.0,0.052219,0.485348,0.7790
1,0.997992,0.383603,0.051316,0.0135,0.0,0.901,0.727273,0.0763,0.494026,1.0,0.0,0.047678,0.344019,0.0767
2,0.997992,0.383603,0.051316,0.0135,0.0,0.901,0.727273,0.0763,0.494026,1.0,0.0,0.047678,0.344019,0.0767
3,0.606426,0.758097,0.018374,0.2200,0.0,0.000,0.454545,0.1190,0.627609,0.0,0.0,0.958720,0.439086,0.8800
4,0.998996,0.790486,0.032538,0.1300,0.0,0.887,0.090909,0.1110,0.708887,0.0,0.0,0.095562,0.442470,0.7200


In [25]:
index_artist[2176] , artist_index['Queen']

('Queen', 2176)

In [26]:
df['artists'] = artists_names

In [32]:
df.to_csv('df_cleaned__by_artist.csv')

# Model

- Create a generator with True/ Fake samples
    - Fake samples: Choose a random vector and assign a label that is not its true artist label (1/-1)
- Create an embedding layer for the artists and pass the remaining features to a FC layer
- Train the model to see if it can detect fakes (BCE)
- User inputs song
    - Featurize the song using spotify api
    - Input the features into model(Artist in dict | Artist not in dict)
    - Works for Artist in dict, if artist not in dict???

In [69]:
import torch

In [70]:
class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, df, cols, false_pct=0.3):
        'Initialization'
        self.df = df
        self.false_pct = false_pct
        self.cols = cols

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.df)

    def __getitem__(self, index):
        'Generates one sample of data'
        true_label = 1
        fake_label = -1
        print(index)


        return torch.tensor(self.df[self.cols].iloc[index]), [0]

In [71]:
# Parameters
params = {'batch_size': 2,
          'shuffle': True,
          'num_workers': 6}

In [72]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [77]:
training_set = Dataset(df, list(df.columns), false_pct=0.5)
# training_generator = torch.utils.data.DataLoader(training_set, **params)

In [80]:
train_dl = torch.utils.data.DataLoader(training_set, batch_size=1, shuffle=True, num_workers=0)
_iter = iter(train_dl)

test_sample, labels = next(_iter)

14041


In [81]:
test_sample

tensor([[0.9608, 0.3947, 0.0275, 0.0072, 0.0000, 0.0000, 0.9091, 0.0923, 0.5247,
         1.0000, 0.0700, 0.0615, 0.3073, 0.3300]], dtype=torch.float64)